## Instalar as bibliotecas necessárias para a análise

In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import numpy as np

##### Ler o arquivo CSV

In [ ]:
data = pd.read_csv("AccidentsPortugues.csv")
data.head()

In [ ]:
# Visualizar as informações sobre as colunas do dataset
data.info()

In [ ]:
# Visualizar os valores nulos do dataset
print(data.isnull().sum())

#### Há valores nulos nas colunas "br", "km", "tipo_acidente", "regional" e "delegacia". Esses valores precisam ser tratados entendendo a causa da invalidez de cada um

In [ ]:
# A coluna "br" deve ser string, mas foi interpretada automaticamente como float (pois os nomes das BRs são números).
# Antes de converter os dados, é preciso fazer uma cópia do dataset
data_backup = data.copy()

# Com o backup feito, posso alterar a coluna "br" para string
data['br'] = data['br'].astype(str).str.split('.').str[0]

# Visualizando os dados nulos após a alteração
print(data.isnull().sum())

In [ ]:
# Restam poucos nulos, então vou verificar a quantidade total de linhas para me certificar que a remoção dos nulos não afetará a estatística.

print(data.shape)

In [ ]:
# Como existem 463.152 linhas, vou remover todos os nulos restantes.
data = data.dropna()

# Visualizando a informação de nulos
print(data.isnull().sum())

#visualizando o total de linhas no dataset, estes serão os dados analisados
print(data.shape)

In [ ]:
# Novamente, visualizando os dados da tabela para iniciar a análise a respeito dos acidentes
data.head()

In [ ]:
# Com a visualização, algumas perguntas surgem, para respondê-las, vou utilizar gráficos, e para isso, vou importar a biblioteca de visualização.
import matplotlib.pyplot as plt

# Qual a BR com mais acidentes?
acidentes_br = data["br"].value_counts().nlargest(10)

acidentes_br.plot(kind='bar', title='Acidentes por BR')
plt.xlabel('BR')
plt.ylabel('acidentes')
plt.show()

In [ ]:
# Quantos acidentes têm vítimas feridas, fatais ou não possui vítimas?
class_acidente = data["classificacao_acidente"].value_counts().nlargest(10)

class_acidente.plot(kind='bar', title='Acidentes por classe')
plt.xlabel('Clasificação')
plt.ylabel('acidentes')
plt.show()

In [ ]:
# Qual UF tem mais acidentes?
acidentes_uf = data["uf"].value_counts().nlargest(10)

acidentes_uf.plot(kind='bar', title='Acidentes por UF')
plt.xlabel('UF')
plt.ylabel('Acidentes')
plt.show()

In [ ]:
# Qual dia da semana tem mais acidentes?
acidentes_dia_semana = data["dia_semana"].value_counts().nlargest(10)

acidentes_dia_semana.plot(kind='bar', title='Acidentes por dia da semana')
plt.xlabel('Dia Semana')
plt.ylabel('Acidentes')
plt.show()

In [ ]:
# Qual o tipo de acidente mais comum?
tipo_acidente = data["tipo_acidente"].value_counts().nlargest(10)

tipo_acidente.plot(kind='bar', title='Tipos de acidentes')
plt.xlabel('Tipo')
plt.ylabel('Acidentes')
plt.show()

In [ ]:
# Qual a causa de acidente mais comum?
print("Causas comuns de acidentes")
data["causa_acidente"].value_counts()

In [ ]:
# Quantos acidentes por ano?
data["data_inversa"].value_counts()

In [ ]:
# Para somar o total de acidentes por ano, é necessário modificar o formato da coluna "data_inversa", de string para datetime.
# Antes de alterar, farei outro backup.
data_backup2 = data.copy()

# Agora, vou alterar
data['data_inversa'] = pd.to_datetime(data['data_inversa'])

# Visualizar o tipo de dado na coluna
data.info()

In [ ]:
# Agora posso somar o total de acidentes por ano
acidentes_por_ano = data['data_inversa'].dt.year.value_counts().sort_index()

acidentes_por_ano.plot(kind='bar', title='Acidentes por Ano')
plt.xlabel('Ano')
plt.ylabel('Acidentes')
plt.show()

## Buscando correlações

In [ ]:
data.info()

In [ ]:
# Backup dos dados antes das alterações no dataset
backup3 = data.copy()

In [ ]:
# Para criar o mapa de correlação, preciso instalar o sklearning.preprocessing, que transformará os dados para gerar a visualização.
!pip install sklearn.preprocessing

In [ ]:
# Com o sk instalado, vou importar o LabelEncoder, que armazena o valor numérico que será inserido para cada valor objeto ou categórico
from sklearn.preprocessing import LabelEncoder

# Agora, vou selecionar apenas as colunas categóricas e strings
cat_cols = data.select_dtypes(include=['object', 'category']).columns

# Codificar automaticamente com o LabelEncoder
label_encoders = {}
data_encoded = data.copy()

# Atribuir valor numérico às colunas selecionadas
for col in cat_cols:
    le = LabelEncoder()
    data_encoded[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le  # Guarda os encoders para referência

# Criar o heatmap de correlação (Spearman)
plt.figure(figsize=(20, 15))
sns.heatmap(data_encoded.corr(method='spearman'), 
            annot=True, 
            fmt=".2f", 
            cmap='coolwarm',
            center=0,
            vmin=-1,
            vmax=1)
plt.title('Matriz de Correlação (Spearman)')
plt.show()

In [ ]:
# Já sei quais as BRs ondem ocorrem mais acidentes, mas preciso descobrir quais as BRs que mais têm acidentes fatais.
top_br_mortos = data.groupby('br')['mortos'].sum().nlargest(10)

top_br_mortos.plot(kind='bar', title='Mortos por BR')
plt.xlabel('BR')
plt.ylabel('Mortos')
plt.show()

In [ ]:
# Sabendo que as BRs 101 e 116 têm (cada uma) mais que o dobro de mortes que as demais BRs, quero identificar padrões nos acidentes.
# Para isso, vou filtrar dados para BR-101 e BR-116
brs_criticas = data[data['br'].isin([101, 116])]

# Agrupar por BR e analisar fatores
fatores = ['tipo_acidente', 'condicao_metereologica', 'fase_dia', 'tipo_pista','veiculos']
for fator in fatores:
    print(f"\nDistribuição de {fator} nas BRs 101 e 116:")
    display(pd.crosstab(brs_criticas['br'], brs_criticas[fator], normalize='index'))

In [ ]:
# Alguns insights obtidos com o resultado:

# 1. 91,33% dos acidentes envolvem ATÉ 2 CARROS.
# 2. 49% das mortes ocorrem em PISTAS DUPLAS, 15% em PISTAS MÚLTIPLAS e 36% em PISTAS SIMPLES.
# 3. 53% das mortes ocorrem em PLENO DIA, 35% em PLENA NOITE.
# 4. 53% das mortes ocorrem sob CÉU CLARO, 11% sob CHUVA.
# 5. 21% das mortes vêm da COLISÃO TRASEIRA, 10% vêm de COLISÃO TRANSVERSAL, 11% vêm da SAÍDA DO LEITO CARROÇÁVEL (saída da pista).

In [ ]:
# Agora, quero entender a correlação entre o tipo de acidente e o tipo de pista, para isso, usarei a biblioteca seaborn:
import seaborn as sns

sns.heatmap(
    pd.crosstab(
        data['tipo_pista'], 
        data['tipo_acidente'], 
        values=data['mortos'], 
        aggfunc='sum',
        normalize='index'
    ),
    annot=False,
    fmt=".1%",
    cmap='Reds'
)
plt.title('Proporção de Mortes por Tipo de Pista e Acidente');

In [ ]:
# Insights obtidos com o heatmap:

# 1. A COLISÃO FRONTAL tem correlação FORTE com PISTA SIMPLES.
# 2. O ATROPELAMENTO DE PEDESTRE tem correlação FORTE com PISTAS DUPLAS E MÚLTIPLAS, e FRACA com PISTAS SIMPLES.
# 3. A SAÍDA DO LEITO CARROÇÁVEL tem correlação FORTE com PISTAS SIMPLES E DUPLAS, e FRACA com PISTAS MÚLTIPLAS.

## Fim.